## Neural Network

In [102]:
import torch
import torch.nn as nn #neural net
import torch.nn.functional as F #Functions such as relu
import torch.optim as optim #Parameter optimization functions
import numpy as np
import random #Random shuffling
from scipy.io import loadmat #Loading data

Define the network

In [103]:
class ANN(nn.Module):
    
    def __init__(self):
        #Define the network
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(400, 25)
        self.fc2 = nn.Linear(25,10)
        self.Sigmoid = nn.Sigmoid()
        self.Softmax = nn.Softmax(dim = -1)

    def forward(self, x):
        x = self.Sigmoid(self.fc1(x))
        x = self.Softmax(self.fc2(x))
        return(x)
    
    def loadWeight(self, weights):
        
        fc1 = torch.from_numpy(weights["Theta1"][:,:-1]).float()
        fc1Bias = torch.from_numpy(weights["Theta1"][:,0]).float()
        fc2 = torch.from_numpy(weights["Theta2"][:,:-1]).float()
        fc2Bias = torch.from_numpy(weights["Theta2"][:,0]).float()
        
        print(fc1.shape)
        print(fc1Bias)
        print(fc2Bias)
        
        param = list(self.parameters())
        param[0].data.sub_(fc1)
        param[1].data.sub_(fc1Bias)
        param[2].data.sub_(fc2)
        param[3].data.sub_(fc2Bias)

In [104]:
def loadData(name):
    return(loadmat(name))

In [105]:
data = loadData("ex3data1.mat")
print("Input shape: ", data["X"].shape)
print("Output shape: ", data["y"].shape)

Input shape:  (5000, 400)
Output shape:  (5000, 1)


In [111]:
#Evaluate the network
def evaluate(model, data):
    dataSize = len(data["X"][:,1]) #Total number of samples.
    #Test on the dataset
    #For each of the featureset.
    samples = list(range(dataSize))
    random.shuffle(samples)
    correct = 0

    with torch.no_grad():
        for i in samples:
            features = torch.from_numpy(data["X"][i]).float()
            #Extract the correct value
            target = torch.zeros((10))
            target[data["y"][i]-1] = 1.0
            #Predict
            outputs = classifier(features)
            _, predicted = torch.max(outputs, 0)
            correct += (int(predicted) == data["y"][i]-1)
    return(100*correct/dataSize)
print("Accuracy%: ", evaluate(classifier, data))

Accuracy%:  [94.5]


### Train the Network

In [107]:
#Initialize the neural net
classifier = ANN()
print(classifier)
#Create the optim
optimizer = optim.SGD(classifier.parameters(), lr=0.2)

ANN(
  (fc1): Linear(in_features=400, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=10, bias=True)
  (Sigmoid): Sigmoid()
  (Softmax): Softmax()
)


In [117]:
#Training
dataSize = len(data["X"][:,1]) #Total number of samples.
for epoch in range(10):
    
    lossCount = []
    accuracyCount = []
    correctCount = 0
    #For each of the featureset.
    samples = list(range(dataSize))
    random.shuffle(samples)
    for i in samples:
        #Train with SGD for each set. |
        optimizer.zero_grad()   # zero the gradient
        #Extract a feature i
        features = torch.from_numpy(data["X"][i]).float()
        #Extract the correct value
        target = torch.zeros((10))
        target[data["y"][i]-1] = 1.0
        #Define the MSE loss.
        output = classifier(features)
        loss = nn.MSELoss()(output, target)
        #Backprop the loss to each node.
        loss.backward()
        #Updates our weights using optim SGD.
        optimizer.step()
        #Keep track of loss and accuracy.
        lossCount.append(loss)
    lossCount = np.array(lossCount, dtype=float)
    mean = np.mean(lossCount)
    print("Epoch:"+ str(epoch) + " Avg MSE Loss:", mean, "Ave Accuracy%: ", evaluate(classifier, data))
    


Epoch:0 Avg MSE Loss: 0.0033221883845615564 Ave Accuracy%:  [98.44]
Epoch:1 Avg MSE Loss: 0.003267398532864635 Ave Accuracy%:  [98.48]
Epoch:2 Avg MSE Loss: 0.003225699139590652 Ave Accuracy%:  [98.5]
Epoch:3 Avg MSE Loss: 0.0031578404628805927 Ave Accuracy%:  [98.54]
Epoch:4 Avg MSE Loss: 0.0031113516207628934 Ave Accuracy%:  [98.5]
Epoch:5 Avg MSE Loss: 0.0030710928747006002 Ave Accuracy%:  [98.54]
Epoch:6 Avg MSE Loss: 0.003024588048180694 Ave Accuracy%:  [98.56]
Epoch:7 Avg MSE Loss: 0.0029881792776655933 Ave Accuracy%:  [98.58]
Epoch:8 Avg MSE Loss: 0.0029424303247170542 Ave Accuracy%:  [98.58]
Epoch:9 Avg MSE Loss: 0.0028983883814656707 Ave Accuracy%:  [98.58]


In [9]:
#Update the weights.

### Accuracy of loaded weights

In [62]:
#Load some weights
weights = loadData("ex3weights.mat")
classifier.loadWeight(weights)

torch.Size([25, 400])
tensor([-0.0226, -0.0984,  0.1162, -0.2397, -0.7316, -0.5979,  0.1546, -0.0337,
        -0.4107,  0.0235,  0.2477,  0.2653,  0.0943,  0.2022, -0.2030,  0.1046,
         0.1489, -0.0379, -0.3320, -0.2977, -0.4842, -0.3898, -0.1832, -0.7021,
        -0.3509])
tensor([-0.7610, -0.6179, -0.6893, -0.6783, -0.5966, -0.8779, -0.5275, -0.7490,
        -0.6665, -0.4609])
